# SIwB lab04

Autorzy: Zespół: Max Adamski, Sławomir Gilewski, Patryk Jedlikowski, Mikołaj Sienkiewicz

In [1]:
#!pip install -U xmltodict

In [4]:
#!cp -r ../input/siwblab4/RNA-Puzzles ./RNA-Puzzles

In [5]:
import os
import pandas as pd
import numpy as np
import xmltodict
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from math import ceil
from functools import lru_cache

In [6]:
assert os.path.exists('RNA-Puzzles/'), 'You should extract SIwB-lab-04-data.zip in this directory'
temp_dir = 'cache'
if not os.path.exists(temp_dir):
    os.mkdir(temp_dir)
# pliki .pdb - reprezentacja kartezjańska (https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html)
# pliki .tor - reprezentacja torsyjna

In [7]:
def read_xml(path):
    with open(path) as f:
        data = xmltodict.parse(f.read())
    scores = {}
    errors = {}
    for structure in data['measureScores'].values():
        for description in structure:
            filename = description['description']['filename']
            file_err = description['description']['errors']
            if file_err: errors[filename] = file_err['error']
            scores[filename] = float(description['score'])
    
    return scores, errors

scores, errors = read_xml('RNA-Puzzles/pz01/1_solution_0_rpr_A_10_C-rmsd.xml')
scores

In [8]:
pdb_columns = 'type index atom residue chain residue_sequence x y z occupancy temperature_factor element'.split()
        
def preprocess_pdb(df):
    for val in 'ACUG':
        df['residue_'+val] = df.residue.str.contains(val).astype(int)
    for val in 'CNOP':
        df['atom_'+val] = df.atom.str.contains(val).astype(int)
    df = df.drop(columns='index atom chain residue_sequence'.split())
    df = df.drop(columns=['type', 'residue', 'element']).fillna(0).apply(pd.to_numeric)
    return df

def parse_atom(l):
    return [
        'ATOM',
        int(l[6:11].strip()), # atom serial number
        l[12:16].strip(), # atom name
        #l[16], # alternate location indicator (don't care)
        l[17:20].strip(), # residue name
        l[21].strip(), # chain id
        int(l[22:26]), # residue sequence number
        #l[26], # code for insertion of residues (don't care)
        float(l[30:38]), # x coordinate (A)
        float(l[38:46]), # y coordinate (A)
        float(l[46:54]), # z coordinate (A)
        float(l[54:60]), # occupancy
        float(l[60:66]), # temperature factor
        #l[72:76].strip(), # segment identifier (don't care)
        l[13] #l[76:78].strip(), # element symbol
    ]

def read_pdb(path, *, preprocess=False):
    records = []
    with open(path) as f:
        for line in f.readlines():
            head = line[:3]
            #if head == 'TER':
            #    records.append(['TER'])
            if head == 'ATO':
                records.append(parse_atom(line))
            #else:
            #    assert False, f'unknown record {line}'
    df = pd.DataFrame(records, columns=pdb_columns)
    if preprocess:
        df = preprocess_pdb(df)
    return df
    
df = read_pdb('RNA-Puzzles/pz01/1_solution_0_rpr_A_10_C.pdb', preprocess=True)
df

In [9]:
from glob import glob
import os

prefix = 'RNA-Puzzles'

data = []
for i in range(10):
    puzzle = f'pz{i+1:02}'
    set = 'test' if puzzle == 'pz10' else 'train'
    for rmsd_xml in glob(f'{prefix}/{puzzle}/*-rmsd.xml'):
        rna = os.path.basename(rmsd_xml).strip('-rmsd.xml')
        scores, errors = read_xml(rmsd_xml)
        for solution, score in scores.items():
            solution_path = os.path.join(prefix, puzzle, rna, solution)
            pdb_path = os.path.join(prefix, puzzle, rna + '.pdb')
            data.append(dict(set=set, rna=rna, rna_pdb=pdb_path, solution=solution_path, score=score))

data = pd.DataFrame(data)

for paths in [data.solution, data.rna_pdb]:
    for path in paths:
        assert os.path.exists(path), f'{path} doesn\'t exist!'

data

In [10]:
from sklearn.model_selection import train_test_split
data_train_valid, data_test = data[data.set == 'train'], data[data.set == 'test']
data_train, data_valid = train_test_split(data_train_valid, test_size=0.05, random_state=42)
data_valid.set = 'valid'
print('tr size', len(data_train), len(data_train) / len(data))
print('vl size', len(data_valid), len(data_valid) / len(data))
print('te size', len(data_test), len(data_test) / len(data))

In [11]:
X_tr = data_train.to_dict('records')
X_vl = data_valid.to_dict('records')
X_te = data_test.to_dict('records')
X_tr[0]

In [12]:
import torch as T
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_sequence, pad_sequence

class RNNModel(nn.Module):
    def __init__(self, pdb_features=13, rnn_size=128, rnn_units=8, hidden_size=128, bidirectional=True):
        super().__init__()
        directions = 2 if bidirectional else 1
        self.encoder = nn.GRU(batch_first=True, bidirectional=bidirectional, num_layers=rnn_units,
                              input_size=pdb_features, hidden_size=rnn_size)
        self.lin1 = nn.Linear(2*rnn_units*directions*rnn_size, hidden_size)
        self.lin2 = nn.Linear(hidden_size, hidden_size)
        self.linear = [self.lin1, self.lin2]
        self.output = nn.Linear(hidden_size, 1)
        
    def forward(self, x1, x2, *, batch_size):
        _, h1 = self.encoder(x1)
        _, h2 = self.encoder(x2)
        h1 = h1.permute(1, 0, 2).reshape(batch_size, -1)
        h2 = h2.permute(1, 0, 2).reshape(batch_size, -1)
        x = T.cat([h1, h2], axis=1)
        for layer in self.linear:
            x = F.elu(layer(x))
        x = T.relu(self.output(x))
        return x

def minibatches(permutation, batch_size):
    batch_count = int(ceil(len(permutation) / batch_size))
    result = []
    for batch_idx in range(batch_count):
        result.append(permutation[batch_idx*batch_size:][:batch_size])
    return result

def param_count(x):
    return sum(T.numel(p) for p in x.parameters())

def prog(x, y):
    res = f'{x}/{y}'
    res = '0'*(len(str(y)) - len(str(x))) + res
    return res
    
def batch(dataset, indices=None):
    if indices is None: indices = range(dataset)
    X1 = pack_sequence([read_input(dataset[i]['rna_pdb']) for i in indices], enforce_sorted=False).to(dev)
    X2 = pack_sequence([read_input(dataset[i]['solution']) for i in indices], enforce_sorted=False).to(dev)
    y = T.Tensor([dataset[i]['score'] for i in indices]).unsqueeze(1).to(dev)
    return X1, X2, y

@lru_cache(32)
def read_input(pdb):
    return T.Tensor(read_pdb(pdb, preprocess=True).to_numpy()).to(dev)

dev = T.device('cuda' if T.cuda.is_available() else 'cpu')
model = RNNModel().to(dev)
lr = 1e-4
optim = T.optim.AdamW(model.parameters(), lr=lr)
epoch_count = 10
train_history = []
valid_history = []
batch_size = 32
valid_batch_size = 128
criterion = nn.MSELoss()

print('training model with', param_count(model), 'parameters')
train_batch = 0
for epoch_idx in range(epoch_count):
    train_losses = []
    batches = minibatches(T.randperm(len(X_tr)), batch_size)
    
    # --- TRAINING
    model.train()
    for batch_idx, idxs in enumerate(batches):
        optim.zero_grad()
        X1, X2, y = batch(X_tr, idxs)
        Y = model(X1, X2, batch_size=len(idxs))
        loss = criterion(Y, y)
        loss.backward()
        optim.step()

        train_losses.append(loss.item())
        train_loss = np.mean(train_losses)
        prefix = f'\rtrain epoch {prog(epoch_idx+1, epoch_count)} | batch {prog(batch_idx+1, len(batches))}'
        print(f'{prefix} | train loss = {train_loss:.4f}', end='')
        train_history.append(dict(epoch=epoch_idx, batch=train_batch, batch_loss=loss.item()))

        train_batch += 1
            
    # --- VALIDATION
    model.eval()
    with T.no_grad():
        batches = minibatches(np.arange(len(X_vl)), valid_batch_size)
        Y, y = [], []
        for batch_idx, idxs in enumerate(batches):
            X1, X2, y_batch = batch(X_vl, idxs)
            Y_batch = model(X1, X2, batch_size=len(idxs))
            Y.append(Y_batch)
            y.append(y_batch)
        Y, y = T.cat(Y), T.cat(y)
        
        valid_loss = criterion(Y, y).item()
        valid_rmse = ((Y - y)**2).mean().sqrt().item()
        print(f' | valid loss = {valid_loss:.4f} | valid rmse = {valid_rmse:.4f}', end='')
        valid_history.append(dict(epoch=epoch_idx, train_loss=train_loss, valid_loss=valid_loss, valid_rmse=valid_rmse))

    print()

# 128*8 + 2*128x128x1

In [13]:
# --- TESTING
model.eval()
with T.no_grad():
    batches = minibatches(np.arange(len(X_te)), valid_batch_size)
    Y, y = [], []
    for batch_idx, idxs in tqdm(list(enumerate(batches))):
        X1, X2, y_batch = batch(X_te, idxs)
        Y_batch = model(X1, X2, batch_size=len(idxs))
        Y.append(Y_batch)
        y.append(y_batch)
Y, y = T.cat(Y).cpu().detach().numpy(), T.cat(y).cpu().detach().numpy()
test_rmse = np.sqrt(np.mean((Y - y)**2))
print(test_rmse)

In [42]:
df_train = pd.DataFrame(train_history)
df_valid = pd.DataFrame(valid_history)
plt.rc('figure', dpi=110, figsize=(10, 5))
sns.set_style('white')
sns.set_context('talk')
sns.lineplot(x='epoch', y='batch_loss', label='Training loss', ci=None, data=df_train)
sns.lineplot(x='epoch', y='valid_loss', label='Validation loss', data=df_valid)
plt.legend()
plt.ylabel('MSE Loss')
epoch_list = np.arange(df_train.epoch.max()+1)
plt.xticks(epoch_list, epoch_list+1)
plt.show()

In [43]:
sns.lineplot(x='epoch', y='valid_rmse', data=df_valid)
plt.ylabel('Validation RMSE')
epoch_list = np.arange(df_valid.epoch.max() + 1)
plt.xticks(epoch_list, epoch_list+1)
plt.show()

In [35]:
sns.distplot(y - Y)
plt.xlabel('Test error distribution')
print(np.mean(np.abs(y - Y)))
print(np.sqrt(np.mean((y - Y)**2)))

In [43]:
def free_memory():
    for obj in gc.get_objects():
        try:
            if (T.is_tensor(obj) or T.is_tensor(obj.data)) and obj.is_cuda:
                del obj
        except Exception as e:
            pass
    
    gc.collect()
    T.cuda.empty_cache()

free_memory()